**To use this notebook:** Run one line at a time waiting for each cell to return before running the next cell.

# Menu

Menu items can be made in Jupyterlab with a connection provided back to the item in ipylab.

## Main menu

We can add items to existing menus. But lets first create a new menu and add it to the main menu.

In [ ]:
import ipywidgets as ipw

import ipylab

app = ipylab.app

In [ ]:
t = app.main_menu.create_menu("🌈 MY CUSTOM MENU 🎌")

Menus can be added directly to the shell or nested inside other menus. Let's add this menu to main menu up top.

In [ ]:
menu = t.result()
app.main_menu.add_menu(menu)

See above that the new menu is now added

Lets populate the new menu.

In [ ]:
async def populate_menu(menu):
    await menu.add_item(command="help:about")
    await menu.add_item(type="separator")
    submenu = await app.main_menu.create_menu("My submenu")
    await submenu.add_item(command="notebook:create-console")
    await menu.add_item(submenu=submenu, type="submenu")
    await menu.add_item(command="logconsole:open")

    # Open it
    await app.main_menu.set_property("activeMenu", ipylab.pack(menu), toObject=["value"])
    await app.main_menu.execute_method("openActiveMenu")

In [ ]:
app.to_task(populate_menu(menu))

In [ ]:
menu.activate()

### Built in menus

The built in menus are accessible under `app.main_menu` and can be manipulated in the same way.

In [ ]:
t = app.main_menu.file_menu.add_item(command="logconsole:open")

In [ ]:
app.main_menu.file_menu.add_item(command="logconsole:open")
app.main_menu.file_menu.activate()

## Context menu

The app provides a global context menu. We can add items to the context menu using the same functions.

In [ ]:
t = app.to_task(populate_menu(app.context_menu))

In [ ]:
panel = ipylab.Panel([ipw.HTML("<h3>Right click to open the context menu")])

In [ ]:
panel.add_to_shell(mode=ipylab.InsertMode.split_right)

### Selectors

Selectors are used to discriminate what items are shown in context menus.

Ipylab by default provides a selector using the `vpath` which limits the context menu items to ipylab widgets for the current app.vpath.

The scope of the context menu items can be altered by specifying the selector when adding a new item to the context menu. 

Here we can add a dispose command to allow us to remove any element in the main area by using the .jp-Activity selector. Note a period '.' is required before the CSS class name.


In [ ]:
async def show_id(active_widget: ipylab.ShellConnection):
    id_ = await active_widget.get_property("id")
    await app.dialog.show_dialog("Show id", f"Widget id is {id_}")


t = app.commands.add_command("Show id", show_id)

In [ ]:
t = app.context_menu.add_item(command=t.result(), rank=1000, selector=".jp-Notebook")

## On ready

**Requires per-kernel widget manager**

When the workspace is re-loaded, including when the page is refreshed, all objects in the frontend are lost. IpyWidgets get restored from the backend, but the connected objects are not. 

For this reason, creating anything that uses connections should be defined in a function and then registered with on_ready.

`on_ready` is called with the object that was registered. 

For example using `populate_menus from above:

In [ ]:
async def create_menus(app: ipylab.App):
    menu = await app.main_menu.create_menu("🌈 MY CUSTOM MENU 🎌")
    await app.main_menu.add_menu(menu)
    await populate_menu(menu)
    await populate_menu(app.context_menu)


app.on_ready(create_menus)

Reload the page (F5) ignoring any warnings.

The panel that was in the shell and the menus should have been restored.

Note: May require a per-kernel widget manager. See Readme for details on installation.